# Prepairing chesscom data to use

Working with API, prepairing good forma etc

## Imports and functions

In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime
import json

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# for file working
from os import listdir
from os.path import isfile, join

# own libs
import sys
sys.path.append('//home//roman_vm//my_lib//')

from data_load import *

In [2]:
get_data("""

select *
from chesscom_players

limit 5
""")

,index,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color,year,month,inserted_at
0,0,chess,blitz,180+2,True,1,1/2-1/2,2017-01-05,07:37:43,2017-1,sriakshat1987,Rosolimo,1544,1553,repetition,repetition,0.5,0.5,0,0,D80,https://www.chess.com/openings/Grunfeld-Defense-Zaitsev-Gambit,https://www.chess.com/live/game/1886823199,https://api.chess.com/pub/player/sriakshat1987,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.5,1553,1544,0,b,2017,1,2020-07-15 20:59:35.885724
1,0,chess,blitz,180+2,True,1,1-0,2017-01-05,07:50:47,2017-1,schnell_und_sauber,Rosolimo,1518,1544,win,checkmated,1.0,0.0,0,0,B13,https://www.chess.com/openings/Caro-Kann-Defense-Exchange-Variation-3...cxd5-4.Nf3,https://www.chess.com/live/game/1886834117,https://api.chess.com/pub/player/schnell_und_sauber,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1544,1518,0,b,2017,1,2020-07-15 20:59:35.885724
2,0,chess,blitz,180+2,True,1,1-0,2017-01-05,08:10:18,2017-1,dshutty,Rosolimo,1488,1535,win,checkmated,1.0,0.0,0,0,B18,https://www.chess.com/openings/Caro-Kann-Defense-Classical-Variation-5.Ng3-Bg6-6.N1e2,https://www.chess.com/live/game/1886849449,https://api.chess.com/pub/player/dshutty,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1535,1488,0,b,2017,1,2020-07-15 20:59:35.885724
3,0,chess,blitz,180+2,True,1,0-1,2017-01-05,08:55:29,2017-1,Abhi6789,Rosolimo,1502,1542,timeout,win,0.0,1.0,0,0,B10,https://www.chess.com/openings/Caro-Kann-Defense-Two-Knights-Attack,https://www.chess.com/live/game/1886885066,https://api.chess.com/pub/player/abhi6789,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1542,1502,0,b,2017,1,2020-07-15 20:59:35.885724
4,0,chess,blitz,180+2,True,1,0-1,2017-01-05,09:07:03,2017-1,b0yw0rm,Rosolimo,1484,1549,resigned,win,0.0,1.0,0,0,B18,https://www.chess.com/openings/Caro-Kann-Defense-Classical-Variation-5.Ng3-Bg6-6.Nf3-Nd7,https://www.chess.com/live/game/1886894716,https://api.chess.com/pub/player/b0yw0rm,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1549,1484,0,b,2017,1,2020-07-15 20:59:35.885724


In [19]:
# heplfull instruments

# convert unixtime to time:
# fr_unixtime(1565211491) -> '2019-08-07 20:58:11'
def fr_unixtime(ts):
    from datetime import datetime
    return datetime.utcfromtimestamp(int(ts)).strftime('%Y-%m-%d %H:%M:%S')

# convert integer month number to string API format:
# 1 -> '01' 
# 9 -> '09' 
# 12 > '12'
# 112 -> -1
def get_number(x):
    if x>=10 and x<100:
        return str(x)
    elif x<10:
        return '0'+str(x)
    else:
        return -1
    
# retunr classic Elo propabilities
# elo_prob(2882, 2722) -> 0.7152 (72% chanses Carlsen (2882) to beat Wan Hao (2722))
def elo_prob(rw, rb):
    rw=float(rw)
    rb=float(rb)
    res=1/(1+np.power(10, (rb-rw)/400))
    return res

# working with pgn
# pip install pgn-parser
def get_pgn(text):
    from pgn_parser import pgn, parser
    try:
        game = parser.parse(text, actions=pgn.Actions())
        try:
            score=game.tag_pairs['Result']
        except:
            score='Unknown'  
        try:
            date=game.tag_pairs['Date']
        except:
            date='Unknown'
        try:
            time=game.tag_pairs['UTCTime']
        except:
            time='Unknown'
        try:
            eco=game.tag_pairs['ECO']
        except:
            eco='Unknown'
        try:
            ecourl=game.tag_pairs['ECOUrl']
        except:
            ecourl='Unknown'
        
        res={'score':score, 
                'date': date, 
                'time':time, 
                'ECO':eco, 
                'ECO_url':ecourl}
        
    except:
        res={'score':'Unknown',
                'date': '1970-01-01', 
                'time':'00:00:00', 
                'ECO':'Unknown' ,
                'ECO_url':'Unknown'}
    return res

# functions for predictions
# find col with target user (white or black)
def find_col(target_user, col1, wh_val, bl_val):
    if col1==target_user:
        return wh_val
    else:
        return bl_val
# reverse fun of find_col    
def find_opp(target_user, col1, wh_val, bl_val):
    if col1!=target_user:
        return wh_val
    else:
        return bl_val

In [20]:
# get API data and prepairint it

# get json form site and return string
# exmple get_api_data_to_str('eric', '2014', '01')
def get_api_data_to_str(player, year, month):
    import requests
    response = requests.get('https://api.chess.com/pub/player/'+player+'/games/'+year+'/'+month)
    st=response.text
    return st, response.status_code

# convert api data to pandas for next working
def get_str_data_to_pandas(strng):
    import json
    try:
        js = json.loads(strng)
        df=pd.DataFrame(js)
    except:
        df=pd.DataFrame()      
    return df

# convert multilevel json to table
def prepair_pandas_multilevel_data(df):
    num=len(df)
    new_df=pd.DataFrame()
    for i in range(num):
        b=pd.io.json.json_normalize(df.values[i])
        new_df=pd.concat([new_df, b])
    return new_df 

# get player games in month with preparation
def get_pl_stat(game_stat):    
    # chess result dictionary
    res_dict={
    'win': 1.0,
    'checkmated': 0.0,
    'agreed': 0.5,
    'repetition': 0.5,
    'timeout': 0.0,
    'resigned': 0.0,
    'stalemate': 0.5,
    'lose': 0.0,
    'insufficient': 0.5,
    '50move': 0.5,
    'abandoned': 0.0,
    'kingofthehill': 0.5,
    'threecheck': 0.5,
    'timevsinsufficient': 0.5,
    'bughousepartnerlose': 0.0
    }

    # number of games
    num=len(game_stat)
    
    # cut long and unusefull columns
    game_stat=game_stat[['rules', 'time_class', 'time_control', 'rated',
        'white.@id', 'white.rating', 'white.result', 'white.username',
         'black.@id', 'black.rating', 'black.result', 'black.username',
         'end_time', 'pgn', 
         'url']]
    # score 0.0, 0.5 or 1.0
    game_stat['white.score']=game_stat['white.result'].map(res_dict)
    game_stat['black.score']=game_stat['black.result'].map(res_dict)
    
#     # pgn - long string
    game_stat['pgn']=game_stat['pgn'].apply(get_pgn)
    # result - string like '1-0'
    game_stat['result']=game_stat['pgn'].apply(lambda x: x['score'])
    
    game_stat['date']=game_stat['pgn'].apply(lambda x: x['date'])
    game_stat['time']=game_stat['pgn'].apply(lambda x: x['time'])
    
    # ECO - string like 'B10'
    game_stat['eco']=game_stat['pgn'].apply(lambda x: x['ECO'])
    # ECOurl - url like 'https://www.chess.com/openings/B10-Caro-Kann-Defense-2.Nf3-d5'
    game_stat['eco_url']=game_stat['pgn'].apply(lambda x: x['ECO_url'])
    
#     propability from classic formula
    game_stat['white_elo_forecast']=game_stat[['white.rating', 'black.rating']].apply(lambda x: elo_prob(*x), axis=1)
    game_stat['black_elo_forecast']=game_stat[['black.rating', 'white.rating']].apply(lambda x: elo_prob(*x), axis=1)
    
    # after it np.sum(df['game']) means number of games
    game_stat['game']=1
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    # period - stirng like '2019-08'
    game_stat['period']=game_stat['date'].dt.year.astype('str')+'-'+game_stat['date'].dt.month.astype('str')
        
    game_stat.columns=['rules', 'time_class', 'time_control', 'rated', 'white_url',
       'white.rating', 'white.result', 'white.username', 'black_url',
       'black.rating', 'black.result', 'black.username', 'end_time', 'pgn', 'game_url',
       'white.score', 'black.score', 'result', 'date', 'time', 'eco',
       'eco_url', 'white_elo_forecast', 'black_elo_forecast',
       'game', 'period']
    game_stat=game_stat[[
        'rules', 'time_class', 'time_control', 'rated', 'game',
        'result', 'date', 'time', 'period',
        'white.username', 'black.username',
        'white.rating', 'black.rating', 
        'white.result', 'black.result', 
        'white.score', 'black.score', 
        'white_elo_forecast', 'black_elo_forecast',
        'eco', 'eco_url', 
        'game_url', 'white_url', 'black_url'
    ]]
    
    
    game_stat['date'] = game_stat['date'].astype('datetime64[ns]')
    player=game_stat['white.username'].value_counts().index[0]
    year=game_stat['date'].value_counts().index[0].strftime("%Y")
    month=game_stat['date'].value_counts().index[0].strftime("%m")
    return game_stat

# data for predict current user
def learn_prepair(df, target_user):
    df=df[(df['white.username']==target_user)|(df['black.username']==target_user)]
    df['target_user']=target_user
    df['score']=df[['target_user', 'white.username', 'white.score', 'black.score']].apply(lambda x: find_col(*x), axis=1)
    df['rating']=df[['target_user','white.username', 'white.rating', 'black.rating']].apply(lambda x: find_col(*x), axis=1)
    df['opponent_rating']=df[['target_user', 'white.username', 'white.rating', 'black.rating']].apply(lambda x: find_opp(*x), axis=1)
    df['base_elo_forec']=df[['target_user', 'white.username', 'white_elo_forecast', 'black_elo_forecast']].apply(lambda x: find_col(*x), axis=1)
    df['color']=np.where(df['white.username']==target_user, 'w', 'b')
    
    df['date'] = df['date'].astype('datetime64[ns]')
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    
    return df
    


In [21]:
def get_df(player, year, month):
    response_str, status_code=get_api_data_to_str(player, year, month)
    first_data=get_str_data_to_pandas(response_str)
    second_data=prepair_pandas_multilevel_data(first_data)
    third_data=get_pl_stat(second_data)
    df_to_work=learn_prepair(third_data, player)
    
    return  df_to_work

def insert_df(df, table_name):
    import datetime 
    now=datetime.datetime.now() 
    df['inserted_at']=now
    
    postgresql_engine=get_engine()
    
    df.to_sql(table_name, con=postgresql_engine, if_exists='append')#, index_label='insert_index')    
    print('Ready: ', len(df), ' rows inserted')

In [17]:
year=2020
month=5
player='Rosolimo'
response_str, status_code=get_api_data_to_str(player,str(year), get_number(month))


In [22]:
first_data=get_str_data_to_pandas(response_str)
second_data=prepair_pandas_multilevel_data(first_data)
third_data=get_pl_stat(second_data)
# df_to_work=learn_prepair(third_data, player)

/home/roman_vm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [23]:
third_data.sample(3)

,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url
0,bughouse,blitz,300,False,1,Unknown,1970-01-01,00:00:00,1970-1,Rosolimo,matveyakin,1556,1020,checkmated,win,0.0,1.0,0.956289,0.043711,Unknown,Unknown,https://www.chess.com/live/game/4796675521,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/matveyakin
0,chess,bullet,120+1,True,1,0-1,2020-05-13,16:07:26,2020-5,Stefano_Dungherese,Rosolimo,1395,1603,checkmated,win,0.0,1.0,0.231948,0.768052,C10,https://www.chess.com/openings/French-Defense-Rubinstein-Blackburne-Defense,https://www.chess.com/live/game/4850894480,https://api.chess.com/pub/player/stefano_dungherese,https://api.chess.com/pub/player/rosolimo
0,chess,blitz,180+2,True,1,0-1,2020-05-19,17:09:02,2020-5,Rosolimo,bobbyonly1,1545,1563,resigned,win,0.0,1.0,0.474119,0.525881,E61,https://www.chess.com/openings/Kings-Indian-Defense-Normal-Variation-4.Nf3,https://www.chess.com/live/game/4878711209,https://api.chess.com/pub/player/rosolimo,https://api.chess.com/pub/player/bobbyonly1


In [7]:
year=2020
month=7
player='Rosolimo'
op=get_df(player,str(year), get_number(month))
insert_df(op, 'chesscom_players')

('Ready: ', 18, ' rows inserted')


In [12]:
get_data("""
--drop table chesscom_players
SELECT * FROM chesscom_players limit 5

""")

,index,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color,year,month,inserted_at
0,0,chess,blitz,180+2,True,1,1/2-1/2,2017-01-05,07:37:43,2017-1,sriakshat1987,Rosolimo,1544,1553,repetition,repetition,0.5,0.5,0,0,D80,https://www.chess.com/openings/Grunfeld-Defense-Zaitsev-Gambit,https://www.chess.com/live/game/1886823199,https://api.chess.com/pub/player/sriakshat1987,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.5,1553,1544,0,b,2017,1,2020-07-15 20:59:35.885724
1,0,chess,blitz,180+2,True,1,1-0,2017-01-05,07:50:47,2017-1,schnell_und_sauber,Rosolimo,1518,1544,win,checkmated,1.0,0.0,0,0,B13,https://www.chess.com/openings/Caro-Kann-Defense-Exchange-Variation-3...cxd5-4.Nf3,https://www.chess.com/live/game/1886834117,https://api.chess.com/pub/player/schnell_und_sauber,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1544,1518,0,b,2017,1,2020-07-15 20:59:35.885724
2,0,chess,blitz,180+2,True,1,1-0,2017-01-05,08:10:18,2017-1,dshutty,Rosolimo,1488,1535,win,checkmated,1.0,0.0,0,0,B18,https://www.chess.com/openings/Caro-Kann-Defense-Classical-Variation-5.Ng3-Bg6-6.N1e2,https://www.chess.com/live/game/1886849449,https://api.chess.com/pub/player/dshutty,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1535,1488,0,b,2017,1,2020-07-15 20:59:35.885724
3,0,chess,blitz,180+2,True,1,0-1,2017-01-05,08:55:29,2017-1,Abhi6789,Rosolimo,1502,1542,timeout,win,0.0,1.0,0,0,B10,https://www.chess.com/openings/Caro-Kann-Defense-Two-Knights-Attack,https://www.chess.com/live/game/1886885066,https://api.chess.com/pub/player/abhi6789,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1542,1502,0,b,2017,1,2020-07-15 20:59:35.885724
4,0,chess,blitz,180+2,True,1,0-1,2017-01-05,09:07:03,2017-1,b0yw0rm,Rosolimo,1484,1549,resigned,win,0.0,1.0,0,0,B18,https://www.chess.com/openings/Caro-Kann-Defense-Classical-Variation-5.Ng3-Bg6-6.Nf3-Nd7,https://www.chess.com/live/game/1886894716,https://api.chess.com/pub/player/b0yw0rm,https://api.chess.com/pub/player/rosolimo,Rosolimo,1.0,1549,1484,0,b,2017,1,2020-07-15 20:59:35.885724


In [11]:
year_range=range(2017, 2021)
user_list=['Rosolimo']
for user in user_list:
    # status to visulise
    print('Loading of games from user ', user, ' started')
    for year in year_range:
        print('Year ', year, ' is beginning')
        for month in range(1,13):
            now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(now, ' : month ', month, ' started')
            try:
                insert_df(get_df(user,str(year), get_number(month)), 'chesscom_players')
            except:
                print('Error')

('Loading of games from user ', 'Rosolimo', ' started')
('Year ', 2017, ' is beginning')
('2020-07-15 20:59:30', ' : month ', 1, ' started')
('Ready: ', 158, ' rows inserted')
('2020-07-15 20:59:35', ' : month ', 2, ' started')


/home/roman_vm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/roman_vm/.local/lib/python2.7/site-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Ready: ', 62, ' rows inserted')
('2020-07-15 20:59:38', ' : month ', 3, ' started')
('Ready: ', 94, ' rows inserted')
('2020-07-15 20:59:41', ' : month ', 4, ' started')
('Ready: ', 201, ' rows inserted')
('2020-07-15 20:59:47', ' : month ', 5, ' started')
('Ready: ', 56, ' rows inserted')
('2020-07-15 20:59:49', ' : month ', 6, ' started')
('Ready: ', 55, ' rows inserted')
('2020-07-15 20:59:51', ' : month ', 7, ' started')
('Ready: ', 84, ' rows inserted')
('2020-07-15 20:59:53', ' : month ', 8, ' started')
('Ready: ', 88, ' rows inserted')
('2020-07-15 20:59:56', ' : month ', 9, ' started')
('Ready: ', 77, ' rows inserted')
('2020-07-15 20:59:59', ' : month ', 10, ' started')
('Ready: ', 76, ' rows inserted')
('2020-07-15 21:00:01', ' : month ', 11, ' started')
('Ready: ', 16, ' rows inserted')
('2020-07-15 21:00:02', ' : month ', 12, ' started')
('Ready: ', 32, ' rows inserted')
('Year ', 2018, ' is beginning')
('2020-07-15 21:00:03', ' : month ', 1, ' started')
('Ready: ', 54, '

In [8]:
# save data butch from API to file and create navigation file
def save_data(player, year, month):
    # main API function
    data_string, code=get_api_data_to_str(player, year, month)
    
    filenamepath='data/'+str(player)+'_'+str(year)+'_'+str(month)+'.json'
    
    # write main data file
    with open(filenamepath, 'w') as outfile:
        json.dump(data_string, outfile)
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # write status file
    status_string=str(player)+';'+str(year)+';'+str(month)+';'+str(now)+';'+str(code)+';'+filenamepath+';'+data_string[0:180]
    text_file = open('log/'+str(player)+'_'+str(year)+'_'+str(month)+'.csv', 'w')
    text_file.write(status_string)
    text_file.close()

In [9]:
# read all json in folder and concatenate DataFrames
def read_files(path):
    # reading
    l=[]
    cntr=0
    import os
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".csv"):
                 l.append(os.path.join(root, file))
            cntr=cntr+1
    print('Total', cntr, 'files founded')

    # merging
    i=0
    df=pd.DataFrame()
    for link in l:
        try:
            dfb=pd.read_csv(link)
        except:
            dfb=pd.DataFrame()
        df=pd.concat([df, dfb])
        
    df=df.drop_duplicates()
    return df

In [10]:
# get players list by countries
def get_county_players_list(country):
    import requests
    response = requests.get('https://api.chess.com/pub/country/'+country+'/players')
    st=response.text
    if response.status_code !=200:
        print(response.status_code)
        print(st[0:3000])
        
    import datetime
    now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
    try:
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write(st)
        file.close()
    except Exception:
        error_st=response.status_code+st[0:3000]
        file = open('player_lists/players_list_'+country+'_'+now+'.json','w') 
        file.write( error_st)
        file.close()


In [11]:
# load fata from file and convert it to usefull DataFrame
def load_games(file):
    with open(file) as json_file:
        jsf = json.load(json_file)
    jss = json.loads(jsf)
    first_df=pd.DataFrame(jss)
    
    game_stat=prepair_pandas_multilevel_data(first_df)
    res=get_pl_stat(game_stat)
    
    return res

In [12]:
# the function to launch
def get_data(user_list, year_range):
    # user cicle
    for user in user_list:
        # status to visulise
        print('Loading of games from user ', user, ' started')
        for year in year_range:
            print('Year ', year, ' is beginning')
            for month in range(1,13):
                now=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print(now, ' : month ', month, ' started')
                try:
                    save_data(user,str(year), get_number(month)) 
                except:
                    print('Error')
                    # write error string
                    error_string=str(user)+';'+str(year)+';'+str(month)+';'+str(now)
                    text_file = open('error/'+str(user)+'_'+str(year)+'_'+str(month)+'.csv', 'w')
                    text_file.write(error_string)
                    text_file.close()
                    

## Start working

In [13]:
# %%time
# # user_list=['Rosolimo', 'Akash22', 'smcwhtdtmc']
# user_list=['Rosolimo']
# year_range=range(2013,2020)

# get_data(user_list, year_range)

Loading of games from user  Rosolimo  started
Year  2013  is beginning
2020-01-19 20:25:26  : month  1  started
2020-01-19 20:25:33  : month  2  started
2020-01-19 20:25:34  : month  3  started
2020-01-19 20:25:34  : month  4  started
2020-01-19 20:25:35  : month  5  started
2020-01-19 20:25:35  : month  6  started
2020-01-19 20:25:36  : month  7  started
2020-01-19 20:25:37  : month  8  started
2020-01-19 20:25:38  : month  9  started
2020-01-19 20:25:43  : month  10  started
2020-01-19 20:25:50  : month  11  started
2020-01-19 20:25:56  : month  12  started
Year  2014  is beginning
2020-01-19 20:26:06  : month  1  started
2020-01-19 20:26:16  : month  2  started
2020-01-19 20:26:28  : month  3  started
2020-01-19 20:26:36  : month  4  started
2020-01-19 20:26:40  : month  5  started
2020-01-19 20:26:46  : month  6  started
2020-01-19 20:26:51  : month  7  started
2020-01-19 20:26:58  : month  8  started
2020-01-19 20:27:05  : month  9  started
2020-01-19 20:27:10  : month  10  starte